In [ ]:
import sys
import os
# Need to add parent folder to SYS PATH in order to import packages.
# This is meant to be used only 1 folder inside parent. It won't work if it's inside another folder.
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np

In [ ]:
from inwards_tasks.ecap_dashboard import ECapDashboard
from inwards_tasks.ecap_dashboard import ReportedRun

ed = ECapDashboard()

In [ ]:
ed.import_cedant_info(filepath='CEDANT_INFO.txt')

ed.import_run_data(so_report_filepath='SO_REPORTING_6821.txt',
                   epi_filepath='EPI_6821.txt')

ed.import_run_data(so_report_filepath='SO_REPORTING_6788.txt',
                   epi_filepath='EPI_6788.txt')

ed.import_run_data(so_report_filepath='SO_REPORTING_6754.txt',
                   epi_filepath='EPI_6754.txt')

ed.import_run_data(so_report_filepath='SO_REPORTING_6724.txt',
                   epi_filepath='EPI_6724.txt')

In [ ]:
rr = ReportedRun('SO_REPORTING_6854.txt')

In [ ]:
df = rr.get_data(column_filter={'ULTIMATE_ISO_COUNTRY': 'RUS'})
df = df.loc[~df['MODEL_SUB_TYPE'].isin(['CI_POL_UNK', 'CI_POL_KN'])].copy()

In [ ]:
px.scatter(
    df,
    x='ULTIMATE_POD',
    y='EXP_GROSS_OF_RETRO',
    color='MODEL_SUB_TYPE',
)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

for i in df.MODEL_SUB_TYPE.unique():
    if 'POL' not in i:
        df_sub = df.loc[df['MODEL_SUB_TYPE'] == i]
        fig.add_trace(
            go.Histogram(
                x=df_sub['ULTIMATE_POD'],
                y=df_sub['EXP_GROSS_OF_RETRO'],
                histfunc="sum",
                name=f'Sum of TPE-{i}',
            ),
            row=1, col=1
        )

fig.add_trace(
    go.Histogram(
        x=df['ULTIMATE_POD'],
        y=df['EXP_GROSS_OF_RETRO'],
        histfunc="count",
        name='Count of Buyers'
    ),
    row=2, col=1
)

# fig.add_trace(
#     go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
#     row=2, col=1
# )
fig.update_xaxes(tickformat='.2%', row=1, col=1)
fig.update_xaxes(tickformat='.2%', row=2, col=1)
#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.write_html('test.html', include_plotlyjs ='cdn')

In [ ]:
ed.datasets[-1].get_data().to_clipboard(index=False)

In [ ]:
df_old = ed.datasets[-2].get_data().groupby('CUSTOMER_ID', as_index=False).sum()
df_new = ed.datasets[-1].get_data().groupby('CUSTOMER_ID', as_index=False).sum()

df_old['POD'] = df_old['POD_WA_HELPER'] / df_old['EXP_GROSS_OF_RETRO'].sum()
df_new['POD'] = df_new['POD_WA_HELPER'] / df_old['EXP_GROSS_OF_RETRO'].sum()

df_old.merge(df_new, on='CUSTOMER_ID', how='outer', suffixes=('-OLD', '-NEW'))

In [ ]:
cust_id = '08296C1 01'

ed.get_ecap(column_filter={'CUSTOMER_ID': cust_id})#.to_clipboard()
df = pd.DataFrame()
df['ECAP'] = ed.get_ecap(column_filter={'CUSTOMER_ID': cust_id})
df['PD'] = ed.get_pod(column_filter={'CUSTOMER_ID': cust_id})
df['TPE'] = ed.get_tpe(column_filter={'CUSTOMER_ID': cust_id})
df['EPI'] = ed.get_epi(customer_id=cust_id)
df['ECAP/TPE'] = df['ECAP'] / df['TPE']
df['ECAP/EPI'] = df['ECAP'] / df['EPI']
df#.to_clipboard(index=True)

In [ ]:
ed.get_ecap_movement('bond')


In [ ]:
df_old = ed.datasets[0].get_data(column_filter={'CUSTOMER_ID': '09270C1 07'})
df_new = ed.datasets[1].get_data(column_filter={'CUSTOMER_ID': '09270C1 07'})

In [ ]:
df_new.to_clipboard()

In [ ]:
df_new.columns

In [ ]:
cols_to_use = [
    'ALIAS_ID',
    'ULTIMATE_ID',
    'ULTIMATE_NAME',
    'ULTIMATE_POD',
    'EXP_GROSS_OF_RETRO',
    'EC_CONSUMPTION_ND',
    'ULTIMATE_RATING_TYPE',
    'ULTIMATE_RATING'
]


In [ ]:
df_old[cols_to_use].merge(df_new[cols_to_use], on=['ALIAS_ID', 'ULTIMATE_NAME'])

In [ ]:
ed.datasets[-1].get_data(column_filter={'CUSTOMER_ID': '02718B1 01'})

In [ ]:
df = ed.datasets[-4].get_data(column_filter={'CUSTOMER_ID': '02718B1 01'})
df.groupby('MODEL_SUB_TYPE').sum()['EXP_GROSS_OF_RETRO'].to_clipboard()

In [ ]:
import plotly.express as px
df = ed.datasets[-1].get_data(column_filter={'CUSTOMER_ID': '02718B1 01'})
df = df.loc[df['ALIAS_ID'] != '02718B1 01']

fig = px.scatter(df,
                   x="ULTIMATE_POD",
                   y="EXP_GROSS_OF_RETRO",
                   hover_data={'EXP_GROSS_OF_RETRO': ':.4s',
                                'ULTIMATE_ID': True})

fig.update_xaxes(tickformat='.2%')
fig.show()


In [ ]:
ed.cedant_info[ed.cedant_info['Cedant'].str.contains('UNIPOL')]

In [ ]:
ed.get_ecap(column_filter={'CUSTOMER_ID': '09270C1 07'})

In [ ]:
df_model_type = ed.datasets[-1].data[['CONTRACT_ID', 'MODEL_TYPE']].drop_duplicates()

df_model_type.merge(ed.datasets[-1].epi_by_contract, on='CONTRACT_ID', how='left')

In [ ]:
ed.datasets[-1].get_data(column_filter={'CUSTOMER_ID': '09270C1 07'})['EC_CONSUMPTION_ND'].sum()

In [ ]:
ed.get_pod(column_filter={'CUSTOMER_ID': '09270C1 07'})

In [ ]:
ed.get_ecap_movement(model_type='bond')

In [ ]:
df_old = ed.datasets[-2].data
df_old = df_old.loc[df_old['CUSTOMER_ID'] == '09976C1 03']
df_old.groupby(['ULTIMATE_ID', 'ULTIMATE_NAME'], as_index=False).sum()

# Testing out plot ideas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import List, Union

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
df_cred = pd.read_csv('credit_by_cedant_6854.txt', sep='\t')
df_bond = pd.read_csv('bond_by_cedant_6854.txt', sep='\t')

df_bond_top = df_bond.nlargest(n=26, columns='ECAP (€m)')
# Removed Cauciones because it was distorting the Y axis
df_bond_top = df_bond_top[df_bond_top['Cedant'] != 'ASEG DE CAUCIONES (R. RE)'].copy()

df_cred_top = df_cred.nlargest(n=25, columns='ECAP (€m)')

## Bond

In [ ]:
fig_bond_ecap = make_subplots(specs=[[{"secondary_y": True}]])

fig_bond_ecap.add_trace(
    go.Bar(x=df_bond_top['Cedant'],
           y=df_bond_top['ECAP (€m)'],
           name='ECap',
           hovertemplate='<br>'.join([
               "ECAP: %{y:.4s}<br>",
               "<extra></extra>"
           ])),
)

fig_bond_ecap.add_trace(
    go.Bar(x=df_bond_top['Cedant'],
           y=df_bond_top['EPI (€m)'],
           name='EPI',
           hovertemplate='<br>'.join([
               "EPI: %{y:.4s}<br>",
               "<extra></extra>"
           ]),
           visible='legendonly'),
)

fig_bond_ecap.add_trace(
    go.Scatter(x=df_bond_top['Cedant'],
               y=df_bond_top['ECAP / EPI'],
               name='ECap / EPI ratio',
               mode='lines+markers',
               hovertemplate='<br>'.join([
                   "<b>ECap/EPI ratio:</b> %{y}",
                   "<extra></extra>"
               ])),
    secondary_y=True,
)

fig_bond_ecap.update_layout(
    yaxis2_tickformat='.0%',
    template='plotly_white',
    title="Top 30 Cedants by ECap (bond only)",
    hovermode='x'
)

fig_bond_ecap.update_yaxes(secondary_y=True, showgrid=False, zeroline=False)

fig_bond_ecap.write_html(file='bond_by_ecap.html', include_plotlyjs='cdn')


In [ ]:
fig_bond_tpe = make_subplots(specs=[[{"secondary_y": True}]])

fig_bond_tpe.add_trace(
    go.Bar(x=df_bond_top['Cedant'],
           y=df_bond_top['TPE (€m)'],
           name='TPE',
           hovertemplate='<br>'.join([
               "TPE: %{y:.4s}<br>",
               "<extra></extra>"
           ]),
           text=df_bond_top['TPE (€m)'],
           textposition='outside'),
)

fig_bond_tpe.add_trace(
    go.Scatter(x=df_bond_top['Cedant'],
               y=df_bond_top['ECAP / TPE'],
               name='ECap/TPE ratio',
               mode='lines+markers',
               hovertemplate='<br>'.join([
                   "<b>ECap/TPE ratio:</b> %{y:.2%}",
                   "<extra></extra>"
               ])),
    secondary_y=True,
)

fig_bond_tpe.update_layout(
    yaxis2_tickformat='.1%',
    template='plotly_white',
    hovermode='x'
)

fig_bond_tpe.update_yaxes(
    secondary_y=True,
    showgrid=False,
    zeroline=False
)

fig_bond_tpe.update_traces(
    texttemplate='%{text:.3s}'
)

fig_bond_tpe.update_yaxes(
    range=[0, 11e8],
    secondary_y=False
)

fig_bond_tpe.write_html(file='bond_by_tpe.html', include_plotlyjs='cdn')


## Credit Section

In [ ]:
fig_cred_ecap = make_subplots(specs=[[{"secondary_y": True}]])

fig_cred_ecap.add_trace(
    go.Bar(x=df_cred_top['Cedant'],
           y=df_cred_top['ECAP (€m)'],
           name='ECap',
           hovertemplate='<br>'.join([
               "ECAP: %{y:.4s}<br>",
               "<extra></extra>"
           ])),
)

fig_cred_ecap.add_trace(
    go.Bar(x=df_cred_top['Cedant'],
           y=df_cred_top['EPI (€m)'],
           name='EPI',
           hovertemplate='<br>'.join([
               "EPI: %{y:.4s}<br>",
               "<extra></extra>"
           ]),
           visible='legendonly'),
)

fig_cred_ecap.add_trace(
    go.Scatter(x=df_cred_top['Cedant'],
               y=df_cred_top['ECAP / EPI'],
               name='ECap / EPI ratio',
               mode='lines+markers',
               hovertemplate='<br>'.join([
                   "<b>ECap/EPI ratio:</b> %{y}",
                   "<extra></extra>"
               ])),
    secondary_y=True,
)

fig_cred_ecap.update_layout(
    yaxis2_tickformat='.0%',
    template='plotly_white',
    title="Top 30 Cedants by ECap (bond only)",
    hovermode='x'
)

fig_cred_ecap.update_yaxes(secondary_y=True, showgrid=False, zeroline=False)

fig_cred_ecap.write_html(file='cred_by_ecap.html', include_plotlyjs='cdn')


In [ ]:
fig_cred_tpe = make_subplots(specs=[[{"secondary_y": True}]])

fig_cred_tpe.add_trace(
    go.Bar(x=df_cred_top['Cedant'],
           y=df_cred_top['TPE (€m)'] / 1e6,
           name='TPE',
           hovertemplate='<br>'.join([
               "TPE: %{y:.4s}<br>",
               "<extra></extra>"
           ]),
           text=df_cred_top['TPE (€m)'] / 1e6,
           textposition='outside'),
)

fig_cred_tpe.add_trace(
    go.Scatter(x=df_cred_top['Cedant'],
               y=df_cred_top['ECAP / TPE'],
               name='ECap/TPE ratio',
               mode='lines+markers',
               hovertemplate='<br>'.join([
                   "<b>ECap/TPE ratio:</b> %{y:.2%}",
                   "<extra></extra>"
               ])),
    secondary_y=True,
)

fig_cred_tpe.update_layout(
    yaxis2_tickformat='.1%',
    yaxis_tickformat=',.0f',
    template='plotly_white',
    hovermode='x'
)

fig_cred_tpe.update_yaxes(
    secondary_y=True,
    showgrid=False,
    zeroline=False
)

fig_cred_tpe.update_traces(
    texttemplate='%{text:,.0f}M'
)

fig_cred_tpe.update_yaxes(
    range=[0, 4e3],
    ticksuffix='M',
    secondary_y=False
)

fig_cred_tpe.write_html(file='cred_by_tpe.html', include_plotlyjs='cdn')
